In [1]:
#Install yfinance
%pip install yfinance

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


     -------------------------------------- 65.3/65.3 kB 220.3 kB/s eta 0:00:00
     ------------------------------------ 112.2/112.2 kB 326.7 kB/s eta 0:00:00


In [8]:
#Import Libraries
import pandas as pd
import yfinance as yf
from datetime import datetime

#Set a Date Range for 6 months before
start_date = datetime.now() - pd.DateOffset(months=6)
end_date = datetime.now()

#List of Tickers for Apple, Microsoft, Netflix, and Google
tickers = ['AAPL', 'MSFT', 'NFLX', 'GOOG']

#Define an empty list 
ds_list = []

#Download the data from Yahoo Finance
for ticker in tickers:
    data_set = yf.download(ticker, start=start_date, end=end_date)
    ds_list.append(data_set)

ds = pd.concat(ds_list, keys=tickers, names=['Ticker', 'Date'])
print(ds.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
AAPL   2023-03-13  147.809998  153.139999  147.699997  150.470001  150.059525   
       2023-03-14  151.279999  153.399994  150.100006  152.589996  152.173737   
       2023-03-15  151.190002  153.250000  149.919998  152.990005  152.572647   
       2023-03-16  152.160004  156.460007  151.639999  155.850006  155.424850   
       2023-03-17  156.080002  156.740005  154.279999  155.000000  154.577164   

                     Volume  
Ticker Date                  
AAPL   2023-03-13  84457100  
       2023-03-14  73695900  
       2023-03-15  77167900  
      

In [9]:
#reset the index in the dataframe
ds.reset_index(inplace=True)
print(ds.head())

  Ticker       Date        Open        High         Low       Close  \
0   AAPL 2023-03-13  147.809998  153.139999  147.699997  150.470001   
1   AAPL 2023-03-14  151.279999  153.399994  150.100006  152.589996   
2   AAPL 2023-03-15  151.190002  153.250000  149.919998  152.990005   
3   AAPL 2023-03-16  152.160004  156.460007  151.639999  155.850006   
4   AAPL 2023-03-17  156.080002  156.740005  154.279999  155.000000   

    Adj Close    Volume  
0  150.059525  84457100  
1  152.173737  73695900  
2  152.572647  77167900  
3  155.424850  76161100  
4  154.577164  98944600  


In [23]:
#Performance analysis of in the stock market for past 3 months

#import plotly
import plotly.express as px

#Create a line chart in dark mode
px.defaults.template = "plotly_dark"

#Create a line chart
figStckMktAn = px.line(ds, x='Date', 
               y='Close', 
              color='Ticker', 
              title="Stock Market Performance for the Last 6 Months")
figStckMktAn.show()

In [27]:
#Faceted area chart for six months
figFctArCht = px.area(ds, x='Date', y='Close', color='Ticker',
              facet_col='Ticker',
              labels={'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
              title='Stock Prices for Apple, Microsoft, Netflix, and Google')
figFctArCht.show()

In [13]:
#Analyze moving averages
ds['MA10'] = ds.groupby('Ticker')['Close'].rolling(window=10).mean().reset_index(0, drop=True)
ds['MA20'] = ds.groupby('Ticker')['Close'].rolling(window=20).mean().reset_index(0, drop=True)

for ticker, group in ds.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA10', 'MA20']])

Moving Averages for AAPL
           MA10        MA20
0           NaN         NaN
1           NaN         NaN
2           NaN         NaN
3           NaN         NaN
4           NaN         NaN
..          ...         ...
122  183.800999  180.350000
123  183.445000  180.318500
124  183.624998  180.328999
125  183.699998  180.407500
126  183.335999  180.262000

[127 rows x 2 columns]
Moving Averages for GOOG
           MA10        MA20
381         NaN         NaN
382         NaN         NaN
383         NaN         NaN
384         NaN         NaN
385         NaN         NaN
..          ...         ...
503  134.476001  132.184501
504  134.775000  132.487001
505  135.453000  132.836501
506  136.158000  133.215001
507  136.649001  133.458501

[127 rows x 2 columns]
Moving Averages for MSFT
           MA10        MA20
127         NaN         NaN
128         NaN         NaN
129         NaN         NaN
130         NaN         NaN
131         NaN         NaN
..          ...         ...
249  327.

In [24]:
#Visualize moving averages
for ticker, group in ds.groupby('Ticker'):
    figMovAvg = px.line(group, x='Date', y=['Close', 'MA10', 'MA20'], 
                  color_discrete_sequence=['red', 'blue', 'green'],
                  title=f"{ticker} Moving Averages")
    figMovAvg.show()

In [25]:
#Analyze volatility of the companies
ds['Volatility'] = ds.groupby('Ticker')['Close'].pct_change().rolling(window=10).std().reset_index(0, drop=True)
figVol = px.line(ds, x='Date', y='Volatility', 
              color='Ticker', 
              title='Volatility of All Companies')
figVol.show()

In [26]:
#Analyze the correlation between the companies (Netflix and Microsoft)

# create a DataFrame with the stock prices of Netflix and Microsoft
netflix = ds.loc[ds['Ticker'] == 'NFLX', ['Date', 'Close']].rename(columns={'Close': 'NFLX'})
microsft = ds.loc[ds['Ticker'] == 'MSFT', ['Date', 'Close']].rename(columns={'Close': 'MSFT'})
ds_corr = pd.merge(netflix, microsft, on='Date')

# create a scatter plot to visualize the correlation
figCorNFLXnMSFT = px.scatter(ds_corr, x='NFLX', y='MSFT', 
                 trendline='ols', 
                 title='Correlation between Netflix and Microsoft')
figCorNFLXnMSFT.show()